# SavedModel

In [140]:
# Export trained model as savedModel into export folder with exporter_main_v2.py
!python models/research/object_detection/exporter_main_v2.py \
--pipeline_config_path output/pipeline.config \
--trained_checkpoint_dir output/checkpoint \
--output_directory export

python: can't open file 'models/research/object_detection/exporter_main_v2.py': [Errno 2] No such file or directory


# TFLite Model

In [141]:
!python models/research/object_detection/export_tflite_graph_tf2.py \
  --pipeline_config_path output/pipeline.config \
  --trained_checkpoint_dir export/checkpoint \
  --output_directory tflite

python: can't open file 'models/research/object_detection/export_tflite_graph_tf2.py': [Errno 2] No such file or directory


# ONNX Model

In [ ]:
# https://github.com/onnx/tensorflow-onnx
!python -m tf2onnx.convert --saved-model ../tflite/saved_model --output ../tflite/model.onnx

# TFLite Full integer Quantization

In [143]:
# Get name os astropi images
import os
from PIL import Image
import numpy as np 
from numpy import asarray
import tensorflow as tf

data_folder = '../data/'
astro_data = os.listdir(data_folder)
astro_images_file = [file for file in astro_data if os.path.splitext(file)[1] != '.xml'] # remove xml file
astro_images_file.sort()
astro_bbox_file = [file for file in astro_data if os.path.splitext(file)[1] != '.jpg']
astro_bbox_file.sort()

# Get a List of astropi numpy images
astro_images_list = []
for image_file in astro_images_file:
  img_pil = Image.open(data_folder+image_file)
  img_np = asarray(img_pil)
  astro_images_list.append(img_np)

# Get a numpy array of astropi numpy images
astro_images_np = np.array(astro_images_list) # images as numpy array in case needed

astro_image_tensors = [] # images tensor list from numpy array
for astro_image_np in astro_images_list:
  astro_image_tensors.append(tf.expand_dims(tf.convert_to_tensor(
      astro_image_np, dtype=tf.float32), axis=0))

In [145]:
def get_pad_resized_img(size,img_tensor):
  """resize a tensor image to model expected input shape
  Args:
      size (int): size to rescale tensor to width and heigth
      img_tensor (tf.tensor): tf tensor to be rescaled
  Returns:
      resized_img: resized image tensor
  """
  resized_img = tf.image.resize_with_pad(
        img_tensor, size,size,
        antialias=False
    )
  resized_img = tf.cast(resized_img,dtype=tf.uint8)
  return resized_img

In [146]:
astro_image_tensors = [] # images tensor list from numpy array
for astro_image_np in astro_images_list:
  image_tensor = tf.convert_to_tensor(astro_image_np, dtype=tf.float32)
  image_tensor = get_pad_resized_img(320,image_tensor)
  astro_image_tensors.append(image_tensor)

In [147]:
images = astro_image_tensors

In [148]:
def representative_dataset():
  for data in tf.data.Dataset.from_tensor_slices((images)).batch(1).take(8):
    yield [tf.dtypes.cast(data, tf.float32)]

In [ ]:
import tensorflow as tf

saved_model_dir = '../tflite/saved_model'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

In [150]:
with open('../tflite/model_full_quant.tflite', 'wb') as f:
  f.write(tflite_quant_model)